In [1]:
# standard imports
import numpy as np
import pandas as pd
from capston_db_conn import db_conn

In [2]:
conn = db_conn()
print(conn)

<connection object at 0x7f98a0730040; dsn: 'user=postgres password=xxx dbname=capstone host=mads-capstone-db.codt5pemnxi8.us-east-1.rds.amazonaws.com', closed: 0>


In [3]:
rf_sql = """
SELECT *
FROM redfin_county_full;
"""

redfin = pd.read_sql(rf_sql, con=conn, parse_dates=["period_start", "period_end"])
# conn.close()


/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
redfin.head(1)

,county_fips,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,...,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated,county
0,28047,2015-08-01,2015-08-31,30,county,5,1541,f,"Harrison County, MS",None,...,NaN,NaN,NaN,0.0,NaN,NaN,"Gulfport, MS",25060.0,2022-01-09 14:29:56,Harrison County


Also grab monthly unemployment by state, state income tax to add to the redfin data.

In [19]:
unem = """
SELECT date,
       rate,
       state,
       fips
FROM fred_state_monthly_ur
WHERE date >= '1/1/12'::date;
"""

unemployment = pd.read_sql(unem, con=conn, parse_dates='date')

In [21]:
unemployment.head(1)

,date,rate,state,fips
0,2012-01-01,7.1,TX,48


In [ ]:
tax_q = """
SELECT sitl.state,
       sitl.state_fips,
       income_tax_low,
       income_tax_high,
       scitl.corp_income_tax_low,
       scitl.corp_income_tax_high,
       sitl.year
FROM state_income_tax_long sitl
         INNER JOIN state_corp_income_tax_long scitl
                    ON sitl.year = scitl.year AND sitl.state_fips = scitl.state_fips
WHERE sitl.year >= 2012;
"""

tax = pd.read_sql(tax_q, con=conn)

In [ ]:
tax.head()

In [5]:
conn.close()